In [27]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
import statsmodels.api as sm


In [28]:
predictive_processing_dataset = pd.read_excel("All_Features_dataset.xlsx")
scales = pd.read_excel("filled_scales_BEBRASK_RETOS.xlsx")


In [29]:
scales.drop(["SUBJECT_CODE","Age"],axis=1,inplace=True)

common_values = scales['EPRIME_CODE'].unique()
predictive_processing_dataset = predictive_processing_dataset[predictive_processing_dataset['Subject'].isin(common_values)]


In [30]:
metrics_columns = scales.drop("EPRIME_CODE",axis=1).columns.values
predictive_processing_columns = predictive_processing_dataset.drop("Subject",axis=1).columns.values

In [31]:
feature_scales = pd.merge(left = predictive_processing_dataset,right=scales, left_on="Subject",right_on="EPRIME_CODE").drop(["EPRIME_CODE","Subject"],axis=1)

In [32]:
values = list(range(5, 96, 5))

In [33]:
feature_scales[predictive_processing_columns]

,Mean_Rating0,Mean_Rating0_Match,Mean_Rating0_No_Match,Mean_Rating0_Match_Happy,Mean_Rating0_No_Match_Fear,Mean_Rating0_Match_Sad,Mean_Rating0_No_Match_Sad,Mean_Rating0_Match_Fear,Mean_Rating0_No_Match_Fear.1,Dif_Match,...,Intercept_Match_Fear,Intercept_No_Match_Fear,Final_Value_Match,Final_Value_No_Match,Final_Value_Match_Happy,Final_Value_No_Match_Happy,Final_Value_Match_Sad,Final_Value_No_Match_Sad,Final_Value_Match_Fear,Final_Value_No_Match_Fear
0,2.888889,3.555556,1.888889,3.888889,1.500000,3.555556,2.166667,3.222222,2.000000,1.666667,...,2.972222,3.400000,3.492063,1.502924,3.622222,0.428571,3.622222,2.666667,3.422222,1.000000
1,2.533333,3.000000,1.833333,3.333333,1.333333,2.555556,1.833333,3.111111,2.333333,1.166667,...,3.444444,1.333333,2.801587,2.140351,2.933333,1.047619,2.755556,2.047619,2.844444,3.047619
2,2.311111,3.000000,1.277778,3.555556,1.166667,2.666667,1.500000,2.777778,1.166667,1.722222,...,2.944444,0.866667,3.158730,1.304094,3.822222,1.380952,3.133333,1.142857,2.644444,1.380952
3,2.555556,3.148148,1.666667,3.222222,1.000000,3.000000,2.666667,3.222222,1.333333,1.481481,...,3.305556,1.333333,3.584656,1.543860,3.822222,1.000000,3.733333,1.523810,3.155556,1.333333
4,2.266667,2.851852,1.388889,3.666667,1.166667,2.666667,1.333333,2.222222,1.666667,1.462963,...,1.222222,2.666667,3.367725,1.432749,3.200000,1.095238,3.066667,1.761905,3.022222,0.952381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2.088889,2.555556,1.388889,3.111111,1.500000,2.111111,1.333333,2.444444,1.333333,1.166667,...,2.361111,0.333333,2.722222,1.836257,3.711111,1.857143,1.644444,1.619048,2.511111,2.047619
145,2.422222,3.074074,1.444444,3.666667,1.166667,2.777778,1.333333,2.777778,1.833333,1.629630,...,2.861111,0.933333,3.256614,1.654971,3.466667,1.523810,2.511111,0.619048,2.711111,2.476190
146,2.622222,3.222222,1.722222,3.555556,1.000000,2.888889,1.833333,3.222222,2.333333,1.500000,...,3.055556,2.533333,3.063492,1.994152,2.755556,1.000000,2.488889,2.047619,3.355556,2.190476
147,2.577778,3.222222,1.611111,3.777778,1.333333,3.000000,1.666667,2.888889,1.833333,1.611111,...,3.888889,1.933333,2.904762,1.795322,4.111111,1.761905,2.600000,1.523810,2.088889,1.761905


In [34]:

dataframes = {}
all_dictionary = {}
for value in values:
    
# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
    dictionary_distribution = {}

    for metric in metrics_columns:
        dictionary_distribution[metric] = {}
        
        
        for feature in predictive_processing_columns:
            percentile = np.percentile(feature_scales[feature],value)
            if value < 50:
                above_values = feature_scales[feature_scales[feature] > percentile][metric]  
                below_values = feature_scales[feature_scales[feature] <= percentile][metric]  
            else:
                above_values = feature_scales[feature_scales[feature] >= percentile][metric]  
                below_values = feature_scales[feature_scales[feature] < percentile][metric]  

            u_stat, p_value = stats.mannwhitneyu(above_values, below_values, alternative='two-sided')    
            # Calculate means
            high_mean = np.mean(above_values)
            low_mean = np.mean(below_values)
    
            dictionary_distribution[metric][feature] = {'high_mean': high_mean, 'size_high': len(above_values),
                                                       'low_mean': low_mean, 'size_low': len(below_values),
                                                       'p_value': p_value}

    all_dictionary[value] = dictionary_distribution
    df = pd.DataFrame.from_dict({(metric, feature): values
                                 for metric, features in dictionary_distribution.items()
                                 for feature, values in features.items()}, orient='index')

    # Add this DataFrame to the main dictionary of DataFrames
    dataframes[value] = df

In [35]:
p_value_comparison = {}

# Iterate through each metric
for metric in metrics_columns:
    # Initialize a list to store mean p-values per percentile value
    mean_p_values = []

    for percentile in values:
        # Extract p-values for the current percentile dataset
        p_values = dataframes[percentile].xs(metric, level=0)['p_value']
        mean_p_values.append((percentile, p_values.mean()))

    # Find the percentile dataset with the lowest mean p-value
    lowest_p_value_percentile = min(mean_p_values, key=lambda x: x[1])

    # Store the results in the dictionary
    p_value_comparison[metric] = {
        'Percentile with Lowest Mean P-Value': lowest_p_value_percentile[0],
        'Mean P-Value': lowest_p_value_percentile[1]
    }

# Display the comparison results
for metric, result in p_value_comparison.items():
    print(f"\nMetric: {metric}")
    print(f"Percentile with Lowest Mean P-Value: {result['Percentile with Lowest Mean P-Value']}")
    print(f"Mean P-Value: {result['Mean P-Value']}")


Metric: PA
Percentile with Lowest Mean P-Value: 50
Mean P-Value: 0.27726033228271524

Metric: NA.
Percentile with Lowest Mean P-Value: 30
Mean P-Value: 0.4042783185732137

Metric: ERQ_CR
Percentile with Lowest Mean P-Value: 35
Mean P-Value: 0.3399067424929967

Metric: ERQ_ES
Percentile with Lowest Mean P-Value: 10
Mean P-Value: 0.4446241205541413

Metric: UPPSP_NU
Percentile with Lowest Mean P-Value: 80
Mean P-Value: 0.4632761537704402

Metric: UPPSP_PU
Percentile with Lowest Mean P-Value: 70
Mean P-Value: 0.4259165595316258

Metric: UPPSP_SS
Percentile with Lowest Mean P-Value: 30
Mean P-Value: 0.42008721981491887

Metric: UPPSP_PMD
Percentile with Lowest Mean P-Value: 80
Mean P-Value: 0.40399724885660643

Metric: UPPSP_PSV
Percentile with Lowest Mean P-Value: 85
Mean P-Value: 0.3141676967718847

Metric: BIS
Percentile with Lowest Mean P-Value: 90
Mean P-Value: 0.38916990175387867

Metric: BAS_RR
Percentile with Lowest Mean P-Value: 35
Mean P-Value: 0.3964864519540353

Metric: BAS_D


In [36]:
# Initialize a dictionary to store the top three significant counts per metric
top_significant_counts_per_metric = {metric: [] for metric in metrics_columns}

# Iterate through each metric to determine the top three percentiles with the most significant features
for metric in metrics_columns:
    # Create a list to hold the counts for each percentile for this metric
    metric_counts = []

    for percentile, df in dataframes.items():
        # Filter only rows corresponding to the current metric
        p_values = df.xs(metric, level=0)['p_value']

        # Count the number of features with p-values below 0.05
        significant_count = (p_values < 0.05).sum()

        # Add the count to the list
        metric_counts.append((percentile, significant_count))

    # Sort the list of counts in descending order by count
    metric_counts.sort(key=lambda x: x[1], reverse=True)

    # Keep only the top three counts
    top_significant_counts_per_metric[metric] = metric_counts[:3]

# Display the results
for metric, top_counts in top_significant_counts_per_metric.items():
    print(f"\nMetric: {metric}")
    for rank, (percentile, count) in enumerate(top_counts, 1):
        print(f"  {rank}- Highest: Percentile {percentile} with {count} features having significant p-values")




Metric: PA
  1- Highest: Percentile 75 with 12 features having significant p-values
  2- Highest: Percentile 80 with 12 features having significant p-values
  3- Highest: Percentile 70 with 11 features having significant p-values

Metric: NA.
  1- Highest: Percentile 60 with 5 features having significant p-values
  2- Highest: Percentile 20 with 4 features having significant p-values
  3- Highest: Percentile 45 with 4 features having significant p-values

Metric: ERQ_CR
  1- Highest: Percentile 35 with 9 features having significant p-values
  2- Highest: Percentile 55 with 8 features having significant p-values
  3- Highest: Percentile 25 with 7 features having significant p-values

Metric: ERQ_ES
  1- Highest: Percentile 10 with 4 features having significant p-values
  2- Highest: Percentile 75 with 4 features having significant p-values
  3- Highest: Percentile 15 with 3 features having significant p-values

Metric: UPPSP_NU
  1- Highest: Percentile 40 with 3 features having signifi

In [37]:
significant_features_comparison = {}

for metric in metrics_columns:
    # Initialize a list to store counts of significant p-values per percentile value
    significant_counts = []

    for percentile in values:
        # Extract p-values for the current percentile dataset
        p_values = dataframes[percentile].xs(metric, level=0)['p_value']
        # Count the number of p-values below 0.05
        significant_count = (p_values < 0.05).sum()
        significant_counts.append((percentile, significant_count))

    # Find the percentile dataset with the highest count of significant p-values
    highest_significant_percentile = max(significant_counts, key=lambda x: x[1])

    # Store the results in the dictionary
    significant_features_comparison[metric] = {
        'Percentile with Most Significant Features': highest_significant_percentile[0],
        'Count of Significant Features': highest_significant_percentile[1]
    }

# Display the comparison results
for metric, result in significant_features_comparison.items():
    print(f"\nMetric: {metric}")
    print(f"Percentile with Most Significant Features: {result['Percentile with Most Significant Features']}")
    print(f"Count of Significant Features: {result['Count of Significant Features']}")


Metric: PA
Percentile with Most Significant Features: 75
Count of Significant Features: 12

Metric: NA.
Percentile with Most Significant Features: 60
Count of Significant Features: 5

Metric: ERQ_CR
Percentile with Most Significant Features: 35
Count of Significant Features: 9

Metric: ERQ_ES
Percentile with Most Significant Features: 10
Count of Significant Features: 4

Metric: UPPSP_NU
Percentile with Most Significant Features: 40
Count of Significant Features: 3

Metric: UPPSP_PU
Percentile with Most Significant Features: 25
Count of Significant Features: 4

Metric: UPPSP_SS
Percentile with Most Significant Features: 25
Count of Significant Features: 5

Metric: UPPSP_PMD
Percentile with Most Significant Features: 65
Count of Significant Features: 8

Metric: UPPSP_PSV
Percentile with Most Significant Features: 45
Count of Significant Features: 12

Metric: BIS
Percentile with Most Significant Features: 85
Count of Significant Features: 6

Metric: BAS_RR
Percentile with Most Significa

In [38]:
# Initialize a list to store counts of significant p-values per feature per percentile
significant_counts_per_feature = []

for percentile in values:
    # Extract the DataFrame for the current percentile
    df = dataframes[percentile]
    
    # Group by feature only (ignoring the metric)
    significant_count = df['p_value'].groupby(level=1).apply(lambda x: (x < 0.05).sum())
    
    # Store the count results for this percentile
    significant_counts_per_feature.append((percentile, significant_count))

# Print the significant count per feature for each percentile
for percentile, counts in significant_counts_per_feature:
    print(f"\nPercentile: {percentile}")
    print(counts)



Percentile: 5
Cor_Pred_Like                   0
Cor_Pred_Like_Match             0
Cor_Pred_Like_Match_Fear        3
Cor_Pred_Like_Match_Happy       0
Cor_Pred_Like_Match_Sad         0
Cor_Pred_Like_No_Match          1
Cor_Pred_Like_No_Match_Fear     0
Cor_Pred_Like_No_Match_Happy    1
Cor_Pred_Like_No_Match_Sad      0
Dif_Fear                        1
Dif_Happy                       1
Dif_Match                       0
Dif_Sad                         0
Final_Value_Match               0
Final_Value_Match_Fear          2
Final_Value_Match_Happy         3
Final_Value_Match_Sad           1
Final_Value_No_Match            1
Final_Value_No_Match_Fear       0
Final_Value_No_Match_Happy      0
Final_Value_No_Match_Sad        9
Intercept_Match                 1
Intercept_Match_Fear            0
Intercept_Match_Happy           0
Intercept_Match_Sad             1
Intercept_No_Match              0
Intercept_No_Match_Fear         1
Intercept_No_Match_Happy        1
Intercept_No_Match_Sad          0

In [39]:
# Initialize a list to store counts of significant p-values per feature per percentile
significant_counts_per_feature = []

for percentile in values:
    # Extract the DataFrame for the current percentile
    df = dataframes[percentile]
    
    # Group by feature only (ignoring the metric)
    significant_count = df['p_value'].groupby(level=1).apply(lambda x: (x < 0.05).sum())
    
    # Store the count results for this percentile
    significant_counts_per_feature.append((percentile, significant_count))

# Print the significant count per feature for each percentile
for percentile, counts in significant_counts_per_feature:
    print(f"\nPercentile: {percentile}")
    print(sum(counts))



Percentile: 5
47

Percentile: 10
50

Percentile: 15
69

Percentile: 20
75

Percentile: 25
84

Percentile: 30
96

Percentile: 35
93

Percentile: 40
92

Percentile: 45
100

Percentile: 50
86

Percentile: 55
84

Percentile: 60
95

Percentile: 65
79

Percentile: 70
89

Percentile: 75
82

Percentile: 80
98

Percentile: 85
102

Percentile: 90
102

Percentile: 95
95


In [40]:
# Initialize a dictionary to store counts of significant p-values per feature across all percentiles
significant_counts_aggregate = {feature: [] for feature in predictive_processing_columns}

# Iterate through each percentile and count the significant p-values per feature
for percentile in values:
    df = dataframes[percentile]
    
    # Group by feature only (ignoring the metric)
    significant_count = df['p_value'].groupby(level=1).apply(lambda x: (x < 0.05).sum())
    
    # Append the count for each feature to its corresponding list
    for feature, count in significant_count.items():
        significant_counts_aggregate[feature].append((percentile, count))

# Print the results per feature
for feature, counts in significant_counts_aggregate.items():
    print(f"\nFeature: {feature}")
    for percentile, count in counts:
        print(f"  Percentile {percentile}: {count} significant p-values")



Feature: Mean_Rating0
  Percentile 5: 1 significant p-values
  Percentile 10: 1 significant p-values
  Percentile 15: 1 significant p-values
  Percentile 20: 3 significant p-values
  Percentile 25: 1 significant p-values
  Percentile 30: 3 significant p-values
  Percentile 35: 0 significant p-values
  Percentile 40: 2 significant p-values
  Percentile 45: 3 significant p-values
  Percentile 50: 3 significant p-values
  Percentile 55: 3 significant p-values
  Percentile 60: 1 significant p-values
  Percentile 65: 1 significant p-values
  Percentile 70: 1 significant p-values
  Percentile 75: 1 significant p-values
  Percentile 80: 1 significant p-values
  Percentile 85: 0 significant p-values
  Percentile 90: 0 significant p-values
  Percentile 95: 0 significant p-values

Feature: Mean_Rating0_Match
  Percentile 5: 1 significant p-values
  Percentile 10: 0 significant p-values
  Percentile 15: 2 significant p-values
  Percentile 20: 5 significant p-values
  Percentile 25: 1 significant

In [41]:
# Initialize a dictionary to store the maximum count of significant p-values per feature
max_significant_counts = {feature: (None, 0) for feature in predictive_processing_columns}

# Iterate through each percentile and count the significant p-values per feature
for percentile in values:
    df = dataframes[percentile]

    # Group by feature only (ignoring the metric)
    significant_count = df['p_value'].groupby(level=1).apply(lambda x: (x < 0.05).sum())

    # Update maximum count for each feature
    for feature, count in significant_count.items():
        if count > max_significant_counts[feature][1]:
            max_significant_counts[feature] = (percentile, count)

# Print the results showing only the maximum count for each feature
for feature, (max_percentile, max_count) in max_significant_counts.items():
    print(f"\nFeature: {feature}")
    print(f"  Percentile {max_percentile}: {max_count} significant p-values")



Feature: Mean_Rating0
  Percentile 20: 3 significant p-values

Feature: Mean_Rating0_Match
  Percentile 20: 5 significant p-values

Feature: Mean_Rating0_No_Match
  Percentile 10: 3 significant p-values

Feature: Mean_Rating0_Match_Happy
  Percentile 45: 4 significant p-values

Feature: Mean_Rating0_No_Match_Fear
  Percentile 65: 5 significant p-values

Feature: Mean_Rating0_Match_Sad
  Percentile 85: 4 significant p-values

Feature: Mean_Rating0_No_Match_Sad
  Percentile 85: 4 significant p-values

Feature: Mean_Rating0_Match_Fear
  Percentile 5: 1 significant p-values

Feature: Mean_Rating0_No_Match_Fear.1
  Percentile 95: 6 significant p-values

Feature: Dif_Match
  Percentile 80: 10 significant p-values

Feature: Dif_Happy
  Percentile 85: 8 significant p-values

Feature: Dif_Sad
  Percentile 55: 5 significant p-values

Feature: Dif_Fear
  Percentile 95: 3 significant p-values

Feature: Cor_Pred_Like
  Percentile 95: 6 significant p-values

Feature: Cor_Pred_Like_Match
  Percentil

In [42]:
# Initialize a list to store the total count of significant p-values for each percentile
total_significant_counts_per_percentile = []

# Iterate through each percentile and count the significant p-values
for percentile in values:
    df = dataframes[percentile]
    
    # Count the number of significant p-values across all features and metrics
    total_significant_count = (df['p_value'] < 0.05).sum()
    
    # Store the total count with its corresponding percentile
    total_significant_counts_per_percentile.append((percentile, total_significant_count))

# Sort the percentiles by the total count of significant p-values in descending order
sorted_significant_counts = sorted(total_significant_counts_per_percentile, key=lambda x: x[1], reverse=True)

# Print the sorted list
print("\nOrdered list of percentiles by significant p-values count (descending):")
for rank, (percentile, count) in enumerate(sorted_significant_counts, 1):
    print(f"{rank}. Percentile {percentile}: {count} significant p-values")




Ordered list of percentiles by significant p-values count (descending):
1. Percentile 85: 102 significant p-values
2. Percentile 90: 102 significant p-values
3. Percentile 45: 100 significant p-values
4. Percentile 80: 98 significant p-values
5. Percentile 30: 96 significant p-values
6. Percentile 60: 95 significant p-values
7. Percentile 95: 95 significant p-values
8. Percentile 35: 93 significant p-values
9. Percentile 40: 92 significant p-values
10. Percentile 70: 89 significant p-values
11. Percentile 50: 86 significant p-values
12. Percentile 25: 84 significant p-values
13. Percentile 55: 84 significant p-values
14. Percentile 75: 82 significant p-values
15. Percentile 65: 79 significant p-values
16. Percentile 20: 75 significant p-values
17. Percentile 15: 69 significant p-values
18. Percentile 10: 50 significant p-values
19. Percentile 5: 47 significant p-values


In [43]:
dataframes[85][dataframes[85]["p_value"]<0.05]

high_mean  size_high   low_mean  size_low  \
PA       Mean_Rating0_Match_Sad     36.103448         29  32.550000       120   
         Mean_Rating0_No_Match_Sad  35.840000         25  32.717742       124   
         Dif_Happy                  36.375000         24  32.640000       125   
         Cor_Pred_Like_Match_Happy  35.826087         23  32.769841       126   
         Trend_No_Match             35.695652         23  32.793651       126   
...                                       ...        ...        ...       ...   
MSSB_NEG Dif_Fear                    2.608696         23   1.722222       126   
         Trend_Match_Sad             2.692308         26   1.682927       123   
         Final_Value_No_Match_Sad    2.500000         24   1.736000       125   
MSSB_DES Mean_Rating0_Match_Fear     3.500000         24   2.008000       125   
         Trend_No_Match_Fear         1.304348         23   2.420635       126   

                                     p_value  
PA       Mean_Rating0_Match_Sad     0.004800  
         Mean_Rating0_No_Match_Sad  0.033720  
         Dif_Happy                  0.006288  
         Cor_Pred_Like_Match_Happy  0.042310  
         Trend_No_Match             0.048242  
...                                      ...  
MSSB_NEG Dif_Fear                   0.009996  
         Trend_Match_Sad            0.027770  
         Final_Value_No_Match_Sad   0.035993  
MSSB_DES Mean_Rating0_Match_Fear    0.001772  
         Trend_No_Match_Fear        0.016059  

[102 rows x 5 columns]

In [44]:
dataframes[45][dataframes[45]["p_value"]<0.05]

high_mean  size_high   low_mean  \
PA       Mean_Rating0                  34.341463         82  31.895522   
         Mean_Rating0_Match            34.527778         72  32.038961   
         Mean_Rating0_No_Match         34.368421         76  32.068493   
         Mean_Rating0_Match_Happy      34.676471         68  32.037037   
         Mean_Rating0_Match_Sad        34.308642         81  31.970588   
...                                          ...        ...        ...   
MSSB_NEG Intercept_No_Match_Happy       2.161290         62   1.643678   
MSSB_DES Mean_Rating0_No_Match          2.592105         76   1.890411   
         Cor_Pred_Like_No_Match_Happy   2.600000         55   2.042553   
         Intercept_No_Match             2.682927         82   1.716418   
         Intercept_Match_Fear           2.487500         80   1.971014   

                                       size_low   p_value  
PA       Mean_Rating0                        67  0.018508  
         Mean_Rating0_Match                  77  0.010367  
         Mean_Rating0_No_Match               73  0.044310  
         Mean_Rating0_Match_Happy            81  0.009845  
         Mean_Rating0_Match_Sad              68  0.020765  
...                                         ...       ...  
MSSB_NEG Intercept_No_Match_Happy            87  0.019136  
MSSB_DES Mean_Rating0_No_Match               73  0.006535  
         Cor_Pred_Like_No_Match_Happy        94  0.008181  
         Intercept_No_Match                  67  0.001926  
         Intercept_Match_Fear                69  0.022421  

[100 rows x 5 columns]